In [1]:
import copy

from numpy.ma.extras import row_stack
# import numpy as np
from pgmpy.readwrite import BIFReader
import random
import numpy


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/anjaliriedel/Desktop/AI/PycharmProjects/AI2025-Project3/.venv1/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/anjaliriedel/Desktop/AI/PycharmProjects/AI2025-Project3/.venv1/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/anjaliriedel/Desktop/AI/PycharmProjects

In [2]:
#Parameters
GROUP_ID = '04'
ALGORITHM = 've'
NETWORK_NAME = 'networks/child.bif'
REPORT = '[Disease]'
EVIDENCE_LEVEL = 'None'
EVIDENCE = 'Problem5 =Yes'

In [8]:
class Factor:
    def __init__(self, variable, variableValues, combos):
        self.variables = variable
        self.combinations = combos
        self.values = variableValues



    def addVariables(self, vars):
        for variable in vars:
            if variable not in self.variables:
                self.variables.append(variable)

    def cancelOutVariable(self, vars):
        if vars in self.variables:
            self.variables.remove(vars)
        else:
            print('Variable not found, issue with sum out function')

    def setValues(self, values):
        self.values = values


In [4]:
def normalizeDistribution(distribution):
    sum = 0
    ogDist = copy.deepcopy(distribution)
    for value in distribution:
        sum += value
    if sum != 0:
        for i in range(len(distribution)):
            distribution[i] = distribution[i] / sum
    return distribution

In [5]:
def variableElim(query, observedValues, bayesianNetwork):
    variableElimination = []
    for variable in Order(bayesianNetwork.nodes, query):
        variableElimination = makeFactors(variable, observedValues, bayesianNetwork) + variableElimination
        if variable not in observedValues.keys():
            variableElimination = sumOut(variable, variableElimination)
    return normalizeDistribution(pointWise(variableElimination))

def Order(variables, query):
    order = []
    for variable in variables:
        if variable not in query:
            order.append(variable)
    return order

def getAllCombinations(factor):

    combo = [()]

    for num in factor.cardinality:
        result = []
        for i in range(num):
            for nums in combo:
                result.append(nums + (i,))
        combo = result

    return combo

def getValue(possibilities, cpd):

    value = cpd.values
    for num in possibilities:
        value = value[num]

    return value


def makeFactors(variable, evidence, network):
    factors = []

    for factor in network.cpds:
        if variable in factor.state_names:
            values = []
            combos = getAllCombinations(factor)
            for combo in combos:
                probability = getValue(combo, factor)
                values.append({combo:probability})
            factors.append(Factor(factor.variables, values, factor.cardinality))

    return factors

def sumOut(variable, variableElimination):
    factors_with = []
    factors_without = []
    for factor in variableElimination:
        if variable not in factor.variables:
            factors_without.append(factor)
        else:
            factors_with.append(factor)

    combine = pointWise(factors_with)
    index = 0
    for row in combine.values:
        sum = 0
        for column in row:
            sum += column
        combine.values[index] = sum
        index += 1

    combine.cancelOutVariable(variable)

    factors_without.append(combine)
    return factors_without

def pointWise(variableElimination):
    product = []
    temp_factor = Factor(variableElimination[0].variables, variableElimination[0].values)

    for n in range(len(variableElimination) - 1):
        temp_factor.addVariables(variableElimination[n+1].variables)




        row = [[None for _ in range(len(temp_factor.values))] for _ in range(len(variableElimination[n+1].values))]
        for i in range(len(temp_factor.values)):
            for j in range(len(variableElimination[n+1].values)):
                product=variableElimination[n].values[i] * variableElimination[n+1].values[j][i]

                row[j][i] = product
        temp_factor.setValues(row)

    return temp_factor

def gibbsSampling(Network, reportedVars, evidenceVars, numSamples, burnInLength):
    possVals = Network.states
    randomVals = {}

    #Initializing random state values
    for key in possVals.keys():
        if key in evidenceVars.keys():
            randomVals[key] = evidenceVars[key]
        else:
            randomVals[key] = random.choice(possVals[key])

    #initializing counting tables
    countingTables = []
    for var in reportedVars:
        countingTable = []
        for i in range(len(possVals[var])):
            #case where one of the reported vars is given as evidence
            if(var in evidenceVariables):
                countingTable.append('x')
            else:
                countingTable.append(0)
        countingTables.append(countingTable)

    for i in range(numSamples):
        #Need to pick a random (non evidence) node use its markov blanket
        randVar = random.choice(list(Network.nodes))
        while(randVar in evidenceVars.keys()):
            randVar = random.choice(list(Network.nodes))
        mb = getParentsAndChildren(Network, randVar)
        #Calculate probability of variable given its parents
        cpds = Network.get_cpds(randVar)
        valueDistribution = []
        for value in cpds.state_names[randVar]:
            #Calculates P(x_i|parents(X_i))
            parents = mb[0]
            valueArrIndex = cpds.name_to_no[randVar][value]
            valueArr = cpds.values[valueArrIndex]
            for parent in parents:
                parentVal = randomVals[parent]
                parentArrIndex = cpds.name_to_no[parent][parentVal]
                valueArr = valueArr[parentArrIndex]
            varVal = valueArr
            childrenProb = 1
            #Calculates/sums all child probabilities P(y_j | parents(y_j))
            for children in mb[1]:
                childCpds = Network.get_cpds(children)
                parents = Network.get_parents(children)
                childrenArrIndex = childCpds.name_to_no[children][randomVals[children]]
                childrenArr = childCpds.values[childrenArrIndex]
                for parent in parents:
                    if(parent != randVar):
                        parentVal = randomVals[parent]
                    else:
                        parentVal = value
                    parentArrIndex = childCpds.name_to_no[parent][parentVal]
                    childrenArr = childrenArr[parentArrIndex]
                #Fix for 0 probabilities causing the sampling to get stuck
                if(childrenArr == 0):
                    childrenArr = 0.05
                childrenProb *= childrenArr
            valueDistribution.append(varVal*childrenProb)
        #normalized value distribution for randomly selected variable has been found
        valueDistribution = normalizeDistribution(valueDistribution)
        #print("Value distribution:", valueDistribution)
        possibleVals = Network.states[randVar]
        #print("Possible values for variable", randVar, possibleVals)
        chosenVal = numpy.random.choice(possibleVals, p = valueDistribution)
        #print("Chosen value:", chosenVal)
        #setting the variable's randomly (probability-distribution) based value
        randomVals[randVar] = chosenVal
        #counting value for reported variable if not in burn in period
        if(i >= burnInLength and (i%100  == 0)):
            for t in range(len(countingTables)):
                countingIndex = possVals[reportedVars[t]].index(randomVals[reportedVars[t]])
                #case where reported variable is also a part of evidence
                if(countingTables[t][0] != 'x'):
                    countingTables[t][countingIndex] += 1


    x = 0
    for table in countingTables:
        if(table[0] != 'x'):
            table = normalizeDistribution(table)
    print(countingTables)
    return countingTables


def getParentsAndChildren(Network, variable):
    mb = Network.get_markov_blanket(variable)
    nodes = []
    parents = Network.get_parents(variable)
    children = []
    for node in mb:
        nodeParents = Network.get_parents(node)
        if variable in nodeParents:
            children.append(node)
    nodes.append(parents)
    nodes.append(children)
    return nodes

In [ ]:
def createOutput(reportVariables, network, probabilityDistribution):
    fileName = GROUP_ID + '_' + ALGORITHM + '_' + NETWORK_NAME.lstrip('networks/').rstrip('.bif') + '_' + EVIDENCE_LEVEL + '.csv'
    states = network.states
    with(open(fileName, 'w') as file):
        counter = -1
        for var in reportVariables:
            file.write(var)
            counter += 1
            for state in states[var]:
                file.write(",")
                file.write(state)
            file.write("\n")
            for i in range(len(probabilityDistribution[counter])):
                file.write(str(probabilityDistribution[counter][i]))
                if i != len(probabilityDistribution[counter])-1:
                    file.write(",")
                else:
                    file.write("\n")





reader = BIFReader(NETWORK_NAME)
model = reader.get_model()
reportList = REPORT.lstrip('[').rstrip(']')
reportVariables = reportList.split(',')
for var in reportVariables:
    var = var.strip()
evidenceVariables = {}
if(EVIDENCE_LEVEL != "None"):
    splitter = EVIDENCE.split(";")
    for var in splitter:
        #Edge case for variables with '=' characters in their values
        if'"' in var:
            splitter2 = var.split('"')
            evidenceVariables[splitter2[0][:-1].strip()] = splitter2[1].strip()
        else:
            splitter2 = var.split("=")
            evidenceVariables[splitter2[0].strip()] = splitter2[1].strip()



if ALGORITHM == "gibbs":
    probDist = gibbsSampling(model,reportVariables,evidenceVariables, 2100000, 100000)
elif ALGORITHM == "ve":
    variableElim(reportVariables, evidenceVariables, model)
else:
    print("Unrecognized algorithm:", ALGORITHM)

createOutput(reportVariables, model, probDist)

